In [1]:
mutable struct BITSPacket
    version::Int
    type_id::Int
    payload::Any
end

function binary_str2int(str::String)
    out = 0
    for (i,c) in enumerate(reverse(str))
        out += parse(Int, c) * 2^(i-1)
    end
    return out
end

hexdex_table = Dict([
    '0' => "0000",
    '1' => "0001",
    '2' => "0010",
    '3' => "0011",
    '4' => "0100",
    '5' => "0101",
    '6' => "0110",
    '7' => "0111",
    '8' => "1000",
    '9' => "1001",
    'A' => "1010",
    'B' => "1011",
    'C' => "1100",
    'D' => "1101",
    'E' => "1110",
    'F' => "1111",
])

function binary_input(filename)
    output = []
    for line in readlines(filename)
        decoded = ""
        for c in line
            decoded *= hexdex_table[c]
        end
        append!(output, [line=>decoded,])
    end
    return output
end

function parse_packet(input::String)
    version = binary_str2int(input[1:3])
    type_id = binary_str2int(input[4:6])
    index = 7
    if type_id == 4
        payload_str = ""
        while true
            payload_str *= input[index+1:index+5-1] 
            index += 5
            if input[index-5] == '0'
                break
            end
        end
        payload = binary_str2int(payload_str)
    else
        length_type_id = input[index]
        if length_type_id == '0'
            subpacket_length = binary_str2int(input[index+1:index+15])
            subpacket_input = input[index+16:index+16+subpacket_length-1]
            subpackets = []
            while length(subpacket_input) > 0
                subpacket_input, sp = parse_packet(subpacket_input)
                append!(subpackets, [sp,])
            end
            index = index+16+subpacket_length
            payload = subpackets
        elseif length_type_id == '1'
            nsubpackets_left = binary_str2int(input[index+1:index+11])
            subpacket_input = input[index+12:end]
            initial_length = length(subpacket_input)
            subpacket_length = length(subpacket_input)
            index = index + 12
            subpackets = []
            while nsubpackets_left > 0
                subpacket_input, sp = parse_packet(subpacket_input)
                nsubpackets_left -= 1
                append!(subpackets, [sp,])
            end
            index += initial_length - length(subpacket_input)
            payload = subpackets
        end
    end
    return input[index:end], BITSPacket(version, type_id, payload)
end

parse_packet (generic function with 1 method)

In [2]:
function sum_versions(res)
    version_sum = res.version
    if typeof(res.payload) != Int
        for subpacket in res.payload
            version_sum += sum_versions(subpacket)
        end
    end
    return version_sum
end

println("Part 1 Tests:")
println("-"^13)
for (input, bin) in binary_input("test1.txt")
    result = parse_packet(bin)
    println("$input: $(sum_versions(result[2]))")
end
println("-"^13)
input = binary_input("input.txt")[1][2]
println("Part 1 solution: $(sum_versions(parse_packet(input)[2]))")

Part 1 Tests:
-------------


8A004A801A8002F478: 16
620080001611562C8802118E34: 12
C0015000016115A2E0802F182340: 23
A0016C880162017C3686B18A3D4780: 31
-------------


Part 1 solution: 965


In [3]:
function evaluate_packet(p)
    if typeof(p.payload) == Int
        return p.payload
    elseif p.type_id == 0
        return sum([evaluate_packet(ip) for ip in p.payload])
    elseif p.type_id == 1
        return prod([evaluate_packet(ip) for ip in p.payload])
    elseif p.type_id == 2
        return minimum([evaluate_packet(ip) for ip in p.payload])
    elseif p.type_id == 3
        return maximum([evaluate_packet(ip) for ip in p.payload])
    elseif p.type_id == 5
        return Int(evaluate_packet(p.payload[1]) > evaluate_packet(p.payload[2]))
    elseif p.type_id == 6
        return Int(evaluate_packet(p.payload[1]) < evaluate_packet(p.payload[2]))
    elseif p.type_id == 7
        return Int(evaluate_packet(p.payload[1]) == evaluate_packet(p.payload[2]))
    end
end

evaluate_packet (generic function with 1 method)

In [4]:
println("Part 2 Tests:")
println("-"^13)
for (input, bin) in binary_input("test2.txt")
    result = parse_packet(bin)
    println("$input: $(evaluate_packet(result[2]))")
end
println("-"^13)
input = binary_input("input.txt")[1][2]
println("Part 2 solution: $(evaluate_packet(parse_packet(input)[2]))")

Part 2 Tests:
-------------


C200B40A82: 3
04005AC33890: 54
880086C3E88112: 7
CE00C43D881120: 9
D8005AC2A8F0: 1
F600BC2D8F: 0
9C005AC2F8F0: 0
9C0141080250320F1802104A08: 1
-------------
Part 2 solution: 116672213160
